In [1]:
! pip install pandas_profiling==2.6.0

In [6]:
import pandas as pd #for data manipulation 
import seaborn as sc #for data visualization
import matplotlib.pyplot as plt #for data Visualization
import pandas_profiling as pp #For data analysis report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier 
import time

In [40]:
gesture_data = pd.read_csv('https://cainvas-static.s3.amazonaws.com/media/user_data/AyishaR0/gesture25_8.csv')

In [41]:
gesture_data.shape

(211, 601)

In [42]:
gesture_data['Gesture']

0      countDecrease
1      countDecrease
2      countDecrease
3      countDecrease
4      countDecrease
           ...      
206         decrease
207         decrease
208         decrease
209         decrease
210         decrease
Name: Gesture, Length: 211, dtype: object

In [44]:
values = gesture_data['Gesture'].unique()
values

array(['countDecrease', 'countIncrease', 'next', 'light', 'fan',
       'increase', 'switch', 'decrease'], dtype=object)

In [48]:
from sklearn.preprocessing import  LabelEncoder
LE = LabelEncoder()
gesture_data['Gesture'] = LE.fit_transform(gesture_data['Gesture'])
keys = gesture_data['Gesture'].unique()

In [49]:
keys

array([0, 1, 6, 5, 3, 4, 7, 2])

In [50]:
gesture_dict = dict(zip(keys, values))

In [51]:
gesture_dict

{0: 'countDecrease',
 1: 'countIncrease',
 2: 'decrease',
 3: 'fan',
 4: 'increase',
 5: 'light',
 6: 'next',
 7: 'switch'}

In [52]:
model_rf1 = RandomForestClassifier(random_state=0)

In [53]:
X = gesture_data.drop(columns=['Gesture'], axis=1)
Y = gesture_data['Gesture']

In [54]:
X

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600
0,0.079484,9.617382,2.038554,-0.010829,-0.055904,-0.101556,0.039981,9.510845,2.021796,-0.010829,-0.055904,-0.101556,-0.040221,9.427052,2.018205,-0.010829,-0.055904,-0.101556,-0.039024,9.391140,2.098407,-0.010829,-0.055904,-0.101556,0.038784,9.494085,2.165442,-0.010829,-0.055904,-0.101556,0.077090,9.617382,2.313876,-0.010829,-0.055904,-0.101556,0.118986,9.702373,2.414428,-0.010829,...,1.343069,0.033386,-0.042465,-0.063857,0.668432,9.632944,1.450803,0.033386,-0.042465,-0.063857,0.643294,9.636536,1.452000,0.033386,-0.042465,-0.063857,0.596609,9.705964,1.435241,0.033386,-0.042465,-0.063857,0.498451,9.765816,1.369404,0.033386,-0.042465,-0.063857,0.464934,9.788561,1.314339,0.033386,-0.042465,-0.063857,0.479298,9.753846,1.374192,0.033386,-0.042465,-0.063857
1,0.529574,9.599427,2.098407,0.056541,-0.052821,0.114225,0.419446,9.605412,2.181003,0.056541,-0.052821,0.114225,0.382337,9.601821,2.183398,0.056541,-0.052821,0.114225,0.311711,9.655687,2.222900,0.056541,-0.052821,0.114225,0.311711,9.655687,2.222900,-0.003091,0.018272,-0.074096,0.261435,9.665264,2.158259,-0.003091,0.018272,-0.074096,0.273406,9.654491,2.093619,-0.003091,...,1.637543,0.040309,0.130904,0.072628,0.488874,9.443810,1.747671,0.040309,0.130904,0.072628,0.397899,9.404307,1.847026,0.040309,0.130904,0.072628,0.397899,9.404307,1.847026,0.069166,-0.083306,-0.076423,0.496057,9.415081,1.924835,0.069166,-0.083306,-0.076423,0.572668,9.433037,2.008628,0.069166,-0.083306,-0.076423,0.514013,9.458176,2.061298,0.069166,-0.083306,-0.076423
2,1.179572,9.337273,2.668202,-0.020370,-0.012271,-0.038259,1.260971,9.369594,2.696932,-0.020370,-0.012271,-0.038259,1.295686,9.345653,2.836987,-0.020370,-0.012271,-0.038259,1.345962,9.321712,3.091958,-0.020370,-0.012271,-0.038259,1.345962,9.321712,3.091958,-0.073428,-0.103610,0.034580,1.276533,9.307346,3.113505,-0.073428,-0.103610,0.034580,1.250198,9.314529,3.133855,-0.073428,...,2.763967,0.186044,-0.268776,-0.345960,0.859960,9.683220,2.567650,0.186044,-0.268776,-0.345960,0.946147,9.610200,2.390487,0.186044,-0.268776,-0.345960,0.946147,9.610200,2.390487,-0.109905,0.018330,-0.016268,1.006000,9.541968,2.244447,-0.109905,0.018330,-0.016268,1.021561,9.537180,2.181003,-0.109905,0.018330,-0.016268,1.034729,9.522815,2.145092,-0.109905,0.018330,-0.016268
3,1.181966,9.479722,2.826213,-0.053124,-0.034844,-0.092189,1.107749,9.465357,2.817833,-0.053124,-0.034844,-0.092189,0.904251,9.391140,2.791498,-0.053124,-0.034844,-0.092189,0.759408,9.331287,2.786710,-0.053124,-0.034844,-0.092189,0.734270,9.330091,2.750799,-0.053124,-0.034844,-0.092189,0.802501,9.352835,2.793893,-0.053124,-0.034844,-0.092189,0.889886,9.354032,2.831001,-0.053124,...,2.200156,-0.109265,0.122585,-0.361493,-1.080457,9.556334,2.219309,-0.109265,0.122585,-0.361493,-0.990679,9.545559,2.161850,-0.109265,0.122585,-0.361493,-0.921250,9.522815,2.129530,-0.109265,0.122585,-0.361493,-0.872171,9.533588,2.133121,-0.109265,0.122585,-0.361493,-0.801545,9.557529,2.113968,-0.109265,0.122585,-0.361493,-0.714160,9.533588,2.078057,-0.109265,0.122585,-0.361493
4,1.697894,9.017660,3.776670,-0.004836,0.005648,0.044819,1.541081,8.992523,3.642601,-0.004836,0.005648,0.044819,1.541081,8.992523,3.642601,-0.015832,0.059055,0.047030,1.383070,8.972174,3.473817,-0.015832,0.059055,0.047030,1.289701,8.974566,3.344535,-0.015832,0.059055,0.047030,1.289701,9.029631,3.318200,-0.015832,0.059055,0.047030,1.290897,9.081104,3.358900,-0.015832,...,1.043806,0.061719,-0.423703,0.032020,0.124971,9.611397,0.997121,0.061719,-0.423703,0.032020,0.124971,9.611397,0.997121,-0.055974,-0.113558,-0.240484,0.239888,9.580275,1.115629,-0.055974,-0.113558,-0.240484,0.314105,9.583865,1.167102,-0.055974,-0.113558,-0.240484,0.314105,9.612595,1.157526,-0.055974,-0.113558,-0.240484,0.372

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [56]:
print(X_train.shape)
print(X_test.shape)

(168, 600)
(43, 600)


In [25]:
import numpy as np

In [26]:
np.random.seed(42)
start = time.time()
print('Start time is {0: .2f}'.format(start))
param_dist = {'max_depth': [2, 3, 4],
              'bootstrap': [True, False],
                            'max_features': ['auto', 'sqrt', 'log2', None],
                            'criterion': ['gini', 'entropy']}

cv_rf = GridSearchCV(model_rf1, cv = 10,
                     param_grid=param_dist, 
                     n_jobs = 3)
cv_rf.fit(X_train, y_train)
print('Best Parameters using grid search: \n', 
      cv_rf.best_params_)
end = time.time()
print('Time taken in grid search: {0: .2f}'.format(end - start))


Start time is  1599579071.18
Best Parameters using grid search: 
 {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'log2'}
Time taken in grid search:  513.25


In [57]:
# Set best parameters given by grid search 
model_rf1.set_params(criterion = 'gini',
                     max_features = 'log2', 
                     max_depth = 3)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
model_rf1.set_params(warm_start=True, 
                  oob_score=True)

min_estimators = 15
max_estimators = 2000

error_rate = {}

for i in range(min_estimators, max_estimators + 1):
    model_rf1.set_params(n_estimators=i)
    model_rf1.fit(X_train, y_train)
    oob_error = 1 - model_rf1.oob_score_
    error_rate[i] = oob_error


In [58]:

model_rf1.set_params(n_estimators=400,
                     bootstrap = True,
                     warm_start=False,
                     oob_score=False)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [59]:
model_rf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [119]:
labels = list(gesture_dict.values())

In [107]:
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix
y_pred = model_rf1.predict(X_test)
print(len(X_test))
print(accuracy_score(y_test,y_pred))
#print(recall_score(y_test,y_pred))
conf_mat = confusion_matrix(y_test,y_pred)
print(conf_mat)

43
0.9767441860465116
[[6 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0]
 [0 0 6 0 0 0 0 0]
 [0 0 0 2 0 0 0 0]
 [0 0 0 0 7 1 0 0]
 [0 0 0 0 0 5 0 0]
 [0 0 0 0 0 0 7 0]
 [0 0 0 0 0 0 0 6]]


In [121]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test,y_pred, target_names=labels))


Classification Report

               precision    recall  f1-score   support

countDecrease       1.00      1.00      1.00         6
countIncrease       1.00      1.00      1.00         3
         next       1.00      1.00      1.00         6
        light       1.00      1.00      1.00         2
          fan       1.00      0.88      0.93         8
     increase       0.83      1.00      0.91         5
       switch       1.00      1.00      1.00         7
     decrease       1.00      1.00      1.00         6

     accuracy                           0.98        43
    macro avg       0.98      0.98      0.98        43
 weighted avg       0.98      0.98      0.98        43



In [94]:
import pickle
pickle.dump(model_rf1, open('/content/drive/My Drive/AITS/RandomForestModel.pickle', 'wb'))